In [1]:
import pandas as pd
import numpy as np

In [2]:
asset_df=pd.read_csv('/content/asset.csv')

In [3]:
asset_df.head()

,Unnamed: 0,Asset ID,Employee ID,Purchase Date,Last Maintenance Date,CPU Usage (%),Memory Usage (%),Disk Usage (%),Network Activity (MB),OS Version,Software Installed,Age (years),Issue Reported,Incident Description,Time to Failure (hours),Failure Predicted,Resolution Type
0,0,05dcc07b-f32d-4020-9fa5-ff1b2e24a878,63e9ec05-65fc-4123-bf85-1295d8e2947d,2020-04-12,2024-01-16,28,54,97,3351,Windows 10,37,10,Disk error,The software is not compatible with the system...,110,Yes,IT support
1,1,e43a4855-e05d-4de2-81fa-b206ea50e93a,99a8a86e-30d5-4773-8186-c484e6b18e32,2021-01-18,2024-06-11,69,60,24,4901,Windows 10,45,3,Blue screen error,The computer is overheating. The system gets v...,85,No,self
2,2,49540faf-e53f-450c-9881-adb2be550539,85d57faf-2ca4-4b36-bd36-1d1182d433c6,2020-11-10,2024-06-24,87,16,65,1920,Windows 11,44,5,Software compatibility issue,The software is not compatible with the system...,13,No,IT support
3,3,bb69e435-64f0-4c2e-8646-7875ab912b7f,3a08364f-d91b-49f0-8e01-c6214c6e9c1c,2023-01-19,2024-02-12,14,70,22,4347,Windows 11,13,7,Network disconnection,I keep losing my internet connection. This is ...,100,No,self
4,4,f618d7d2-25f2-4527-86f0-00bfd3f4ae9a,c547896b-0b47-4fee-a208-381de516fe4c,2022-04-15,2024-04-19,91,42,46,317,MacOS,50,3,System slow,Battery drains too quickly. I need to plug in ...,46,No,IT support


In [4]:
pip install transformers datasets torch scikit-learn

In [5]:
from sklearn.model_selection import train_test_split

# Reuse the previously defined asset_df
df = asset_df[['Incident Description', 'Issue Reported']]

# Map issues to numerical labels
issue_to_label = {issue: idx for idx, issue in enumerate(df['Issue Reported'].unique())}
df['label'] = df['Issue Reported'].map(issue_to_label)

# Split the data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


<ipython-input-5-5cfdc82a02b2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['Issue Reported'].map(issue_to_label)


In [6]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(issue_to_label))

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['Incident Description'], padding="max_length", truncation=True)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
def classify_incident(description):
    inputs = tokenizer(description, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return predicted_class

issue_to_resolution = {
    "System slow": "self",
    "Application crash": "self",
    "Network disconnection": "self",
    "Blue screen error": "IT support",
    "Disk error": "IT support",
    "Battery issue": "self",
    "Overheating": "IT support",
    "Software compatibility issue": "self"
}

#input
description = "system generates too much heat"
System_id="SYS1001"

predicted_issue = classify_incident(description)
predicted_issue_name = df['Issue Reported'].unique()[predicted_issue]
resolution_type = issue_to_resolution[predicted_issue_name]

print(f"Issue: {predicted_issue_name}, Resolution Type: {resolution_type}")
if resolution_type=="IT support":
  print(f"Ticket raised to IT team for {System_id}....")
else:
  print("Providing Self guide...")


Issue: Software compatibility issue, Resolution Type: self
Providing Self guide...


In [8]:
!pip install --upgrade accelerate
!pip install --upgrade transformers[torch] torch


In [9]:
# Prepare data for regression model
it_support_df = asset_df[asset_df['Resolution Type'] == 'IT support']

X = it_support_df[['CPU Usage (%)', 'Memory Usage (%)', 'Disk Usage (%)', 'Network Activity (MB)', 'Software Installed', 'Age (years)']]
y = it_support_df['Time to Failure (hours)']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Initialize and train the XGBoost regressor
xg_regressor = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1, max_depth=5, alpha=10, n_estimators=100)

xg_regressor.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = xg_regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 3454.488787629229


In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

# Reuse the previously defined asset_df
it_support_df = asset_df[asset_df['Resolution Type'] == 'IT support']

X = it_support_df[['CPU Usage (%)', 'Memory Usage (%)', 'Disk Usage (%)', 'Network Activity (MB)', 'Software Installed', 'Age (years)']]
y = it_support_df['Time to Failure (hours)']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Hyperparameter tuning for XGBoost
param_grid_xgb = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'alpha': [0, 0.01, 0.1, 1, 10],
}

xg_regressor = xgb.XGBRegressor(objective='reg:squarederror')

random_search = RandomizedSearchCV(estimator=xg_regressor, param_distributions=param_grid_xgb,
                                   n_iter=100, scoring='neg_mean_squared_error',
                                   cv=3, verbose=2, random_state=42, n_jobs=-1)

random_search.fit(X_train, y_train)

best_params_xgb = random_search.best_params_
best_xg_regressor = xgb.XGBRegressor(**best_params_xgb)
best_xg_regressor.fit(X_train, y_train)

y_pred_xgb = best_xg_regressor.predict(X_test)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
print(f"Mean Squared Error after XGBoost tuning: {mse_xgb}")


Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Mean Squared Error after XGBoost tuning: 3332.71097104986


In [12]:
import joblib

# Save the trained XGBoost model
model_filename = 'xgboost_model.pkl'
joblib.dump(best_xg_regressor, model_filename)

print(f"Model saved to {model_filename}")


Model saved to xgboost_model.pkl


In [15]:
# Load the model
loaded_model = joblib.load("/content/xgboost_model.pkl")

In [14]:
import math
# Assuming you have new test data in a DataFrame `new_test_df`
# Prepare your new test data similarly to how you prepared the training data
new_test_df = pd.DataFrame({
    'CPU Usage (%)': [20],
    'Memory Usage (%)': [30],
    'Disk Usage (%)': [40],
    'Network Activity (MB)': [100],
    'Software Installed': [5],
    'Age (years)': [2]
})

# Prepare new test data to match training data
X_new = new_test_df[['CPU Usage (%)', 'Memory Usage (%)', 'Disk Usage (%)', 'Network Activity (MB)', 'Software Installed', 'Age (years)']]

# Predict
y_new_pred = loaded_model.predict(X_new)
time_to_failure=math.floor(y_new_pred[0]/24)

# If time to failure is below a certain threshold, send an alert to IT support
threshold = 150  # Example threshold in hours
if time_to_failure < threshold:
    print(f"ALERT: Asset ID {System_id} is predicted to fail in {time_to_failure} days. Sending alert to IT support.")

ALERT: Asset ID SYS1001 is predicted to fail in 4 days. Sending alert to IT support.
